# Face Recognition 

In [ ]:
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import  numpy as np
import cv2
from keras.models import load_model
import smtplib
from email.mime.text import MIMEText


# generating 100 img from 1 img to train 

In [ ]:


from keras.preprocessing.image import array_to_img, ImageDataGenerator, img_to_array, load_img

In [ ]:
GenerateImage= ImageDataGenerator(rotation_range=40, width_shift_range= .2, height_shift_range=.2, shear_range=.2,
                                  zoom_range=.2,horizontal_flip=True, fill_mode='nearest' )

In [ ]:
img= load_img(r"C:\Users\User\Downloads\WhatsApp Image 2024-01-21 at 22.38.38.jpeg")
x=img_to_array(img)
# to generate the image we need to load the img

In [ ]:
x=x.reshape((1,)+ x.shape)
x.shape

In [ ]:
i= 0
for batch in GenerateImage.flow(x, batch_size=1, save_to_dir="d:\preveiw", save_prefix='dd', save_format='JPG'):
    
    i=i+1
    if i>100:
        break

#  model:

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D,Flatten, Dense,Dropout,AvgPool2D

from keras.models import Model
from keras.layers import Dense, Lambda, Flatten, Input
from keras.preprocessing  import image
from keras.models import Sequential
import glob
import keras

In [ ]:
image_size=200

path1=r"D:\face_recognition" # path of folder
cate =["suraj","sakshi","stranger"]# picture containing folder
input_image=[]
for i in cate:
    folders=os.path.join(path1,i)
    label= cate.index(i)
    for image in os.listdir(folders):
        image_path= os.path.join(folders,image)
        image_array= cv2.imread(image_path)
        image_array=cv2.resize(image_array,(image_size,image_size))
        input_image.append([image_array,label])
        

In [ ]:
len(input_image)

In [ ]:
np.random.shuffle(input_image) # every time run the code shuffling is done

In [ ]:
x=[]
y=[]

for x_values,labels in input_image:
    x.append(x_values)
    y.append(labels)

In [ ]:
x= np.array(x)
y=np.array(y)

In [ ]:
y.shape

In [ ]:
x_train = x[:500]
y_train = y[:500]

x_test = x[500:700]
y_test = y[500:700]

In [ ]:
x_train = x_train/255
x_test = x_test/255

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [ ]:
model=Sequential()
model.add(Conv2D(filters=50,kernel_size=(5,5),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu',input_shape=x_train.shape[1:]))
model.add(Dense(128, activation='relu',input_shape=x_train.shape[1:]))
model.add(Dense(128, activation='relu',input_shape=x_train.shape[1:]))
model.add(Dense(3,activation='softmax'))

In [ ]:
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=["accuracy"])

In [ ]:
model.fit(x_train,y_train, epochs=5,batch_size=32)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
pred=model.predict(x_train)
pred_final= pred.argmax(axis=1)

In [ ]:
print(classification_report(y_train,pred_final))

In [ ]:
pred_t=model.predict(x_test)

pred_final_t= pred_t.argmax(axis=1)

In [ ]:
pred_final_t

In [ ]:
print(classification_report(y_test,pred_final_t))

In [ ]:
# model.save("face_recognitio.h5")

#  working

In [ ]:
face_cascade= cv2.CascadeClassifier(r"D:\open cv\HaarCascade\HaarCascade\haarcascade_frontalface_default.xml")
def face_detection(img):
    face_img=img.copy()
    
    face_reacts=face_cascade.detectMultiScale(face_img)# this face is just x,y,height and breath of rectangle
    
    for(x,y,w,h) in face_reacts:
        cv2.rectangle(face_img,(x,y),(x+w,y+h),(255,0,0),3)# frawing rectangle on face
    return face_img

In [ ]:
eye_cascade = cv2.CascadeClassifier(r"C:\Users\Sakshi\HaarCascade\HaarCascade\haarcascade_eye.xml")

def eye_detection(img):
    face_img = img.copy()
    eye_rects =  eye_cascade.detectMultiScale(face_img) 
    for (x,y,w,h) in eye_rects:
        cv2.rectangle(face_img, (x,y), (x+w, y+h), (255,100,10),5)
    return face_img

In [ ]:
from keras.models import load_model
model = load_model('face_recognitio.h5')
results={0:'suraj',1:'sakshi',2:'stranger'}
GR_dict={0:(0,255,255),1:(0,255,0),2:(255,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0)
haarcascade = cv2.CascadeClassifier(r"D:\open cv\HaarCascade\HaarCascade\haarcascade_frontalface_default.xml")
while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(200,200))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,200,200,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)

    cv2.imshow('Liv Camera',   im)
    key = cv2.waitKey(10)
    
    if key == 27: # use the escape key
        break

cap.release()

cv2.destroyAllWindows()

# email code

In [ ]:
model = load_model('face_recognitio.h5')
results = {0: 'suraj', 1: 'sakshi', 2: 'stranger'}
GR_dict = {0: (0, 255, 255), 1: (0, 255, 0),2:(255,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0)
haarcascade = cv2.CascadeClassifier(r"D:\open cv\HaarCascade\HaarCascade\haarcascade_frontalface_default.xml")

%run./email configuration.ipynb

def send_email(message):
    subject = 'Alert: Stranger Detected'
    body = f"Subject: {subject}\n\n{message}"

    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(smtp_username, smtp_password)
        server.sendmail(smtp_username, receiver_email, body)

while True:
    (rval, im) = cap.read()
    im = cv2.flip(im, 1, 1)

    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)

    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f]

        face_img = im[y:y+h, x:x+w]
        rerect_sized = cv2.resize(face_img, (200, 200))
        normalized = rerect_sized / 255.0
        reshaped = np.reshape(normalized, (1, 200, 200, 3))
        reshaped = np.vstack([reshaped])
        result = model.predict(reshaped)

        label = np.argmax(result, axis=1)[0]

        cv2.rectangle(im, (x, y), (x+w, y+h), GR_dict[label], 2)
        cv2.rectangle(im, (x, y-40), (x+w, y), GR_dict[label], -1)
        cv2.putText(im, results[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

        if label == 2:  # Stranger detected
            send_email("Stranger in the house")

    cv2.imshow('Liv Camera', im)
    key = cv2.waitKey(10)

    if key == 27:  # use the escape key
        break

cap.release()
cv2.destroyAllWindows()

# checking without web cam

In [ ]:
# creating function to predict input image
def product_type(img_path):
    # imp labraries
    import cv2
    import matplotlib.pyplot as plt
    import keras
    from keras.models import load_model
    import warnings
    warnings.filterwarnings("ignore")
    
    # loading the models
    model = load_model('face_recognitio.h5')
    
    # processing image
    img = cv2.imread(img_path)
    img = cv2.resize(img, (200, 200))
    img_array = np.array(img)/255
    img_array = img_array.reshape((1, 200, 200, 3))
    
    
    cate = ['suraj', 'sakshi','Stranger']
    
    pred = model.predict(img_array, verbose = 0)
    label = pred.argmax(axis = 1)
    corrected_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(corrected_img)
    plt.title(f"{cate[label[0]]} category")
    plt.axis("off")

In [ ]:
product_type(r"C:\Users\User\Downloads\WhatsApp Image 2023-11-28 at 23.11.37.jpeg")